In [9]:
%cd /home/ltchen/gnnpp
import sys
import os
import pytorch_lightning as L
import torch
import torch_geometric
import json
import wandb

from typing import Tuple
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import scatter
from torch.nn import Linear, ModuleList, ReLU
from torch_geometric.loader import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import AdamW
from pytorch_lightning.loggers import WandbLogger

from models.loss import NormalCRPS
from models.model_utils import MakePositive, EmbedStations
from utils.data import (
    load_dataframes,
    load_distances,
    normalize_features_and_create_graphs,
    rm_edges,
    summary_statistics,
)
from exploration.graph_creation import *
from models.graphensemble.multigraph import *
import shap
from torch_geometric.explain import GNNExplainer

/home/ltchen/gnnpp


In [3]:
leadtime = "24h"

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
DIRECTORY = os.getcwd()
SAVEPATH = os.path.join(DIRECTORY, f"leas_final_models/gnn_run4_{leadtime}/models")
JSONPATH = os.path.join(DIRECTORY, f"trained_models/best_{leadtime}/params.json")

with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config = args_dict

# from gnn_run3 ###############################
max_epoch_list = {
    'g1': 31,
    'g2': 26,
    'g3': 31,
    'g4': 32,
    'g5': 23,
}

[INFO] Loading /home/ltchen/gnnpp/trained_models/best_24h/params.json


In [4]:
dataframes = load_dataframes(mode="eval", leadtime=leadtime)
dataframes = summary_statistics(dataframes)

[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f


In [5]:
graph_name = "g1"
graphs1_train_rf, tests1 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["geo"], edges=[("geo", 100)], sum_stats = True)
graphs1_test_rf, graphs1_test_f = tests1

g1_train_loader = DataLoader(graphs1_train_rf, batch_size=config['batch_size'], shuffle=True)
g1_test_f_loader = DataLoader(graphs1_test_f, batch_size=config['batch_size'], shuffle=False)
g1_test_rf_loader = DataLoader(graphs1_test_rf, batch_size=config['batch_size'], shuffle=False)

train_loader = g1_train_loader
test_f_loader = g1_test_f_loader
test_rf_loader = g1_test_rf_loader
test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs1_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs1_train_rf[0].num_edge_features
max_epochs = max_epoch_list[graph_name]

[INFO] Normalizing features...
fit_transform
transform 1
transform 2


100%|██████████| 730/730 [00:02<00:00, 256.63it/s]


In [7]:
PROJECTNAME = "gnn_run4"
FILENAME = graph_name + "_run_" + leadtime
TRAINNAME = graph_name + "_train_run_" + leadtime
CKPT_PATH = os.path.join(SAVEPATH, TRAINNAME + '.ckpt')
RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/best_{leadtime}/best_{leadtime}_{graph_name}")

multigraph = Multigraph.load_from_checkpoint(
    CKPT_PATH,
    embedding_dim=emb_dim,
    edge_dim=edge_dim,
    in_channels=in_channels,
    hidden_channels_gnn=config['gnn_hidden'],
    out_channels_gnn=config['gnn_hidden'],
    num_layers_gnn=config['gnn_layers'],
    heads=config['heads'],
    hidden_channels_deepset=config['gnn_hidden'],
    optimizer_class=AdamW,
    optimizer_params=dict(lr=config['lr']),
)

multigraph.eval()
trainer = L.Trainer()

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
data_list = ["f", "rf"]
for data, tl in zip(data_list, test_loader):
    preds_list = []
    preds = trainer.predict(model=multigraph, dataloaders=[tl]) # 92 x 976 x 2 forecasts with mu and sigma of 122 stations
    print(preds[0].shape)
    # preds = [prediction.reshape(1, 122, 2).mean(axis=0) for prediction in preds]
    #ACHTUNG - reshape(1, 122, 2) mit 1 statt 5!
    preds = torch.cat(preds, dim=0)
    preds_list.append(preds)

    targets = dataframes[f"test_{data}"][1]
    targets = torch.tensor(targets.t2m.values) - 273.15

    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps: {res.item()}")
    print("#############################################")
    print("#############################################")

    os.makedirs(RESULTPATH, exist_ok=True)

    df = pd.DataFrame(np.concatenate([targets.view(-1, 1), final_preds], axis=1), columns=["t2m", "mu", "sigma"])
    df.to_csv(os.path.join(RESULTPATH, f"{data}_{FILENAME}_results.csv"), index=False)

    # Create Log File ###############################################################
    log_file = os.path.join(RESULTPATH, f"{data}.txt")
    with open(log_file, "w") as f:
        f.write(f"Data: {data}\n")
        f.write(f"Leadtime: {leadtime}\n")
        f.write(f"Final crps: {res.item()}")

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:01<00:00, 53.33it/s]
torch.Size([976, 2])
#############################################
#############################################
final crps: 0.6755105741389072
#############################################
#############################################


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 92/92 [00:01<00:00, 78.75it/s]
torch.Size([976, 2])
#############################################
#############################################
final crps: 0.6981425078190406
#############################################
#############################################


## GNN Explainer

In [ ]:
explainer = GNNExplainer(multigraph, epochs=100, return_type='log_prob')